# split first

In [ ]:
import random
import os 
import shutil
cluster_path = '/home/jiayuan/ADC-cycleGAN/dataset/cluster' # Please change this path to yours.

image_list = os.listdir(os.path.join(cluster_path,'trainMRI'))
test_number = int(len(image_list)*0.1)
sample = random.sample(image_list, test_number)

if not os.path.exists(os.path.join(cluster_path,'testMRI')):
    os.makedirs(os.path.join(cluster_path,'testMRI'))
    os.makedirs(os.path.join(cluster_path,'testCT'))
    
for image in sample:
    shutil.move(os.path.join(cluster_path,'trainMRI',image),os.path.join(os.path.join(cluster_path,'testMRI'),image))
    shutil.move(os.path.join(cluster_path,'trainCT',image),os.path.join(os.path.join(cluster_path,'testCT'),image))



In [ ]:
import os
import numpy as np
from sklearn.cluster import KMeans
import cv2
from imutils import build_montages
import torch.nn as nn
import torchvision.models as models
from PIL import Image
from torchvision import transforms
import glob
import shutil
import joblib

for looptime in range(1,6):
    class Net(nn.Module):
        def __init__(self):
            super(Net, self).__init__()
            resnet50 = models.resnet50(pretrained=True)
            self.resnet = nn.Sequential(resnet50.conv1,
                                        resnet50.bn1,
                                        resnet50.relu,
                                        resnet50.maxpool,
                                        resnet50.layer1,
                                        resnet50.layer2,
                                        resnet50.layer3,
                                        resnet50.layer4)

        def forward(self, x):
            x = self.resnet(x)
            return x

    net = Net().eval()

    image_path = []
    all_images = []
    images = os.listdir(os.path.join(cluster_path,'trainCT'))

    for image_name in images:
        image_path.append(os.path.join(cluster_path,'trainCT',image_name))
    for path in image_path:
        image = Image.open(path).convert('RGB')
        image = transforms.Resize([256,256])(image)
        image = transforms.ToTensor()(image)
        image = image.unsqueeze(0)
        image = net(image)
        image = image.reshape(-1, )
        all_images.append(image.detach().numpy())

    if not os.path.exists(cluster_path+'/center'+str(looptime)):
        os.makedirs(cluster_path+'/center'+str(looptime))

    for clusterN in range(4,5):

        n_clusters = clusterN
        clt = KMeans(n_clusters)
        clt.fit(all_images)
        labelIDs = np.unique(clt.labels_)
#         print(clt.cluster_centers_)
        save_path = cluster_path+'/center'+str(looptime)+'/'+str(n_clusters)+'.pkl'
        # save center model
        joblib.dump(clt, save_path)


        a = []
        b = []
        idxs_list = []
        for labelID in labelIDs:
            idxs = np.where(clt.labels_ == labelID)[0]
            idxs_list.append(idxs)
            idxs = np.random.choice(idxs, size=min(25, len(idxs)),
                replace=False)
            show_box = []
            for i in idxs:
                image = cv2.imread(image_path[i])
                image = cv2.resize(image, (96, 96))
                show_box.append(image)
            montage = build_montages(show_box, (96, 96), (5, 5))[0]

            title = "Type {}".format(labelID)
            a.append(title)
            b.append(montage)
        #     cv2.imshow(title, montage)
        #     cv2.waitKey(0)

            newFilePath=cluster_path+str(looptime)+'/cluster'+str(n_clusters)

            if not os.path.exists(newFilePath):
                os.makedirs(newFilePath)

            count = 1
            for i in idxs_list:
                if not os.path.exists(os.path.join(newFilePath,str(count),'trainCT')):
                    os.makedirs(os.path.join(newFilePath,str(count),'trainCT'))
                    os.makedirs(os.path.join(newFilePath,str(count),'trainMRI'))
                for j in i:
            #         print(image_path[j].split('/')[6])
                    shutil.copy(image_path[j],os.path.join(newFilePath,str(count),'trainCT',image_path[j].split('/')[-1]))
                    shutil.copy(os.path.join(cluster_path,'trainMRI/',image_path[j].split('/')[-1]),os.path.join(newFilePath,str(count),'trainMRI',image_path[j].split('/')[-1]))
                count+=1






# Using K-means cluster test data (CT)

In [ ]:
import os
import numpy as np
from sklearn.cluster import KMeans
import cv2
from imutils import build_montages
import torch.nn as nn
import torchvision.models as models
from PIL import Image
from torchvision import transforms
import glob
import shutil
import joblib

for looptime in range(1,6):
    class Net(nn.Module):
        def __init__(self):
            super(Net, self).__init__()
            resnet50 = models.resnet50(pretrained=True)
            self.resnet = nn.Sequential(resnet50.conv1,
                                        resnet50.bn1,
                                        resnet50.relu,
                                        resnet50.maxpool,
                                        resnet50.layer1,
                                        resnet50.layer2,
                                        resnet50.layer3,
                                        resnet50.layer4)

        def forward(self, x):
            x = self.resnet(x)
            return x

    net = Net().eval()

    image_path = []
    all_images = []
    images = os.listdir(os.path.join(cluster_path,'testCT'))

    for image_name in images:
        image_path.append(os.path.join(cluster_path,'testCT',image_name))
    for path in image_path:
        image = Image.open(path).convert('RGB')
        image = transforms.Resize([256,256])(image)
        image = transforms.ToTensor()(image)
        image = image.unsqueeze(0)
        image = net(image)
        image = image.reshape(-1, )
        all_images.append(image.detach().numpy())

    if not os.path.exists(cluster_path+'/center'+str(looptime)):
        os.makedirs(cluster_path+'/center'+str(looptime))

    for clusterN in range(4,5):

        n_clusters = clusterN
        load_name = cluster_path+'/center'+str(looptime)+'/'+str(n_clusters)+'.pkl'
        clt = joblib.load(load_name)
#         clt.fit(all_images)
        y = clt.predict(all_images)
        labelIDs = np.unique(clt.labels_)
#         print(labelIDs)

        a = []
        b = []
        idxs_list = []
        for labelID in labelIDs:
            idxs = np.where(y == labelID)[0]
            idxs_list.append(idxs)
            idxs = np.random.choice(idxs, size=min(25, len(idxs)),
                replace=False)
            show_box = []
            for i in idxs:
                image = cv2.imread(image_path[i])
                image = cv2.resize(image, (96, 96))
                show_box.append(image)
            montage = build_montages(show_box, (96, 96), (5, 5))[0]

            title = "Type {}".format(labelID)
            a.append(title)
            b.append(montage)
        #     cv2.imshow(title, montage)
        #     cv2.waitKey(0)


            newFilePath=cluster_path+str(looptime)+'/cluster'+str(n_clusters)

            if not os.path.exists(newFilePath):
                os.makedirs(newFilePath)

            count = 1
            for i in idxs_list:
                if not os.path.exists(os.path.join(newFilePath,str(count),'testCT')):
                    os.makedirs(os.path.join(newFilePath,str(count),'testCT'))
                    os.makedirs(os.path.join(newFilePath,str(count),'testMRI'))
                for j in i:
                    shutil.copy(image_path[j],os.path.join(newFilePath,str(count),'testCT',image_path[j].split('/')[-1]))
                    shutil.copy(os.path.join(cluster_path,'testMRI/',image_path[j].split('/')[-1]),os.path.join(newFilePath,str(count),'testMRI',image_path[j].split('/')[-1]))
                count+=1




